In [1]:
# python3 /home/rnb/Projects/rnb-planning/src/pkg/planning/filtering/lattice_model/shared_lattice_predictor.py --rtype panda

# import SharedArray as sa
# ROBOT_TYPE_NAME = 'indy7gripper'
# sa.delete("shm://{}.prepared".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.grasp_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.arm_img".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.rh_vals".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.result".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_in".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.response_out".format(ROBOT_TYPE_NAME))
# sa.delete("shm://{}.query_quit".format(ROBOT_TYPE_NAME))

In [1]:
from __future__ import print_function
import os
RNB_PLANNING_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(RNB_PLANNING_DIR, 'src'))
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir
import subprocess

DATA_PATH = os.path.join(RNB_PLANNING_DIR, "data")
try_mkdir(DATA_PATH)

DAT_ROOT = "stowing-deep"
RES_ROOT = "stowing-deep-result"

TEST_DATA_PATH = os.path.join(DATA_PATH, DAT_ROOT)
try_mkdir(TEST_DATA_PATH)

TEST_RESULT_PATH = os.path.join(DATA_PATH, RES_ROOT)
try_mkdir(TEST_RESULT_PATH)

HEADS = ["plan_time", "length", "MP_count", "failed_MPs", "IK_count", "failed_IKs", "success"]

## Common params

In [2]:
DAT_DIR = '20210907-173519'
CNAME_LIST = ['None', 'Tool', 'ToolReach', 'Full'] #,'Pairwise']
# CNAME_LIST = ['Full'] #,'Pairwise']
VISUALIZE = False
PLAY_RESULT = False
SHOW_STATE = False

TIMEOUT_MOTION = 5
MAX_TIME = 100
MAX_ITER = 100

GRASP_SAMPLE = 30
STABLE_SAMPLE = 50
SEARCH_SAMPLE_RATIO= 1
MAX_SKELETONS = 30

# obj1

In [3]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_1"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

In [4]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

KeyboardInterrupt: 

# obj1 obs5

In [6]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_1_obs3"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

In [7]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 103.4   116.3   5.0     112.0   -------
========== 1 / 20 (eta: 336.7 / 6734.3 s) =========
------- 4.5     7.9     5.3     5.9     -------
========== 2 / 20 (eta: 360.2 / 3602.3 s) =========
------- 8.5     9.4     6.8     5.1     -------
========== 3 / 20 (eta: 390.0 / 2600.3 s) =========
------- 13.6    8.5     5.1     4.9     -------
========== 4 / 20 (eta: 422.2 / 2110.8 s) =========
------- 4.5     5.7     20.1    5.6     -------
========== 5 / 20 (eta: 458.2 / 1832.6 s) =========
------- 4.3     4.1     5.4     4.6     -------
========== 6 / 20 (eta: 476.6 / 1588.5 s) =========
------- 30.5    4.1     9.8     5.1     -------
========== 7 / 20 (eta: 526.1 / 1503.2 s) =========
------- 23.2    5.1     6.0     5.3     -------
========== 8 / 20 (eta: 565.7 / 1414.3 s) =========
------- 103.4   4.8     10.9    5.5     -------
========== 9 / 20 (eta: 690.4 / 1534.2 s) =========
------- 6.4     4.6     8.3     4.9     -------
========== 10 / 20 (eta: 714.6 / 1429.2 s) =========

# obj1 obs5

In [9]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_1_obs5"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

In [10]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 10.7    7.8     5.2     5.2     -------
========== 1 / 20 (eta: 28.9 / 578.5 s) =========
------- 8.8     4.9     4.9     4.5     -------
========== 2 / 20 (eta: 52.1 / 520.7 s) =========
------- 82.3    6.4     5.4     6.2     -------
========== 3 / 20 (eta: 152.4 / 1016.1 s) =========
------- 103.5   37.6    113.1   110.6   -------
========== 4 / 20 (eta: 517.3 / 2586.6 s) =========
------- 104.4   24.3    40.1    110.5   -------
========== 5 / 20 (eta: 796.6 / 3186.6 s) =========
------- 10.1    11.8    36.3    14.8    -------
========== 6 / 20 (eta: 869.7 / 2899.0 s) =========
------- 8.8     5.8     5.1     4.8     -------
========== 7 / 20 (eta: 894.2 / 2554.9 s) =========
------- 108.4   19.9    105.0   71.1    -------
========== 8 / 20 (eta: 1198.5 / 2996.3 s) =========
------- 10.2    5.4     6.7     5.2     -------
========== 9 / 20 (eta: 1226.1 / 2724.6 s) =========
------- 103.8   9.8     104.0   106.9   -------
========== 10 / 20 (eta: 1550.6 / 3101.1 s) =========


# obj3

In [12]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_3"
I_START=0
I_END = 20
N_DAT = I_END-I_START

In [13]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 5.4     5.1     4.6     6.1     -------
========== 1 / 20 (eta: 21.2 / 424.1 s) =========
------- 26.3    3.9     5.5     5.3     -------
========== 2 / 20 (eta: 62.3 / 623.2 s) =========
------- 107.7   9.7     5.3     104.2   -------
========== 3 / 20 (eta: 289.2 / 1928.2 s) =========
------- 13.1    10.0    6.3     4.9     -------
========== 4 / 20 (eta: 323.6 / 1617.9 s) =========
------- 106.3   127.4   7.8     7.6     -------
========== 5 / 20 (eta: 572.7 / 2290.9 s) =========
------- 10.2    5.0     6.7     5.2     -------
========== 6 / 20 (eta: 599.8 / 1999.5 s) =========
------- 16.4    5.9     4.9     4.9     -------
========== 7 / 20 (eta: 631.9 / 1805.5 s) =========
------- 10.1    4.9     4.6     14.3    -------
========== 8 / 20 (eta: 665.8 / 1664.5 s) =========
------- 15.7    9.4     4.5     4.9     -------
========== 9 / 20 (eta: 700.4 / 1556.4 s) =========
------- 15.0    10.3    103.9   4.7     -------
========== 10 / 20 (eta: 834.2 / 1668.3 s) =========
---

# obj3_pole

In [21]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_3_pole"
I_START=0
I_END = 20
N_DAT = I_END-I_START

### test

In [22]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 103.7   103.7   76.7    95.1    -------
========== 1 / 20 (eta: 379.2 / 7584.3 s) =========


KeyboardInterrupt: 

# obj3_hard

In [18]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_3_hard"
I_START=0
I_END = 20
N_DAT = I_END-I_START

### test

In [19]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 103.5   106.3   9.3     103.9   -------
========== 1 / 20 (eta: 323.0 / 6460.4 s) =========
------- 13.3    5.9     7.0     5.9     -------
========== 2 / 20 (eta: 355.1 / 3551.5 s) =========
------- 11.7    5.6     7.8     75.7    -------
========== 3 / 20 (eta: 456.0 / 3039.9 s) =========
------- 11.5    4.8     103.9   7.8     -------
========== 4 / 20 (eta: 584.0 / 2919.9 s) =========
------- 15.0    57.1    105.6   142.8   -------
========== 5 / 20 (eta: 904.4 / 3617.5 s) =========
------- 12.8    104.2   6.2     8.1     -------
========== 6 / 20 (eta: 1035.8 / 3452.5 s) =========
------- 66.9    4.9     5.3     8.2     -------
========== 7 / 20 (eta: 1121.2 / 3203.4 s) =========
------- 103.6   103.5   104.0   103.9   -------
========== 8 / 20 (eta: 1536.2 / 3840.6 s) =========
------- 103.4   103.4   110.1   128.1   -------
========== 9 / 20 (eta: 1981.2 / 4402.7 s) =========
------- 5.5     5.1     8.4     10.4    -------
========== 10 / 20 (eta: 2010.5 / 4021.1 s) ====

# obj1 obs0

In [3]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_1c_obs0"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

### test

In [4]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 12.9    5.6     5.7     5.6     -------
========== 1 / 20 (eta: 29.8 / 596.2 s) =========
------- 5.2     6.7     5.6     6.1     -------
========== 2 / 20 (eta: 53.4 / 533.5 s) =========
------- 7.5     7.2     6.1     5.5     -------
========== 3 / 20 (eta: 79.7 / 531.1 s) =========
------- 7.3     109.1   5.8     5.9     -------
========== 4 / 20 (eta: 207.7 / 1038.7 s) =========
------- 21.4    7.4     5.3     5.4     -------
========== 5 / 20 (eta: 247.3 / 989.1 s) =========
------- 14.6    13.0    7.7     5.8     -------
========== 6 / 20 (eta: 288.4 / 961.2 s) =========
------- 4.6     5.0     14.9    5.5     -------
========== 7 / 20 (eta: 318.4 / 909.8 s) =========
------- 7.6     7.0     5.6     49.9    -------
========== 8 / 20 (eta: 388.6 / 971.5 s) =========
------- 8.9     6.4     49.7    112.7   -------
========== 9 / 20 (eta: 566.3 / 1258.4 s) =========
------- 6.7     4.9     5.1     6.9     -------
========== 10 / 20 (eta: 589.9 / 1179.7 s) =========
------- 5

# obj1 obs3

In [5]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_1c_obs3"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

### test

In [6]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 23.4    5.0     5.3     6.3     -------
========== 1 / 20 (eta: 40.0 / 800.0 s) =========
------- 111.1   6.5     5.3     11.2    -------
========== 2 / 20 (eta: 174.2 / 1741.8 s) =========
------- 27.3    5.3     15.8    33.7    -------
========== 3 / 20 (eta: 256.2 / 1708.0 s) =========
------- 29.7    22.6    6.7     5.8     -------
========== 4 / 20 (eta: 321.0 / 1604.9 s) =========
------- 5.3     9.4     5.3     10.6    -------
========== 5 / 20 (eta: 351.6 / 1406.6 s) =========
------- 23.5    11.7    5.6     5.7     -------
========== 6 / 20 (eta: 398.2 / 1327.4 s) =========
------- 13.5    42.3    9.8     76.0    -------
========== 7 / 20 (eta: 539.8 / 1542.4 s) =========
------- 22.4    6.2     7.1     6.1     -------
========== 8 / 20 (eta: 581.6 / 1453.9 s) =========
------- 5.2     9.6     5.6     6.9     -------
========== 9 / 20 (eta: 608.8 / 1352.9 s) =========
------- 10.0    57.8    31.6    6.4     -------
========== 10 / 20 (eta: 714.5 / 1429.1 s) =========
-

# obj1 obs5

In [7]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_1c_obs5"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

### test

In [8]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 11.8    8.7     6.2     15.2    -------
========== 1 / 20 (eta: 41.9 / 837.9 s) =========
------- 107.2   16.2    104.6   107.5   -------
========== 2 / 20 (eta: 377.4 / 3774.3 s) =========
------- 114.0   18.6    48.3    18.1    -------
========== 3 / 20 (eta: 576.4 / 3843.0 s) =========
------- 9.2     105.1   42.1    65.8    -------
========== 4 / 20 (eta: 798.5 / 3992.6 s) =========
------- 62.4    115.1   7.6     21.6    -------
========== 5 / 20 (eta: 1005.2 / 4020.6 s) =========
------- 11.4    6.1     40.6    6.2     -------
========== 6 / 20 (eta: 1069.4 / 3564.7 s) =========
------- 106.7   130.5   104.8   112.6   -------
========== 7 / 20 (eta: 1524.0 / 4354.3 s) =========
------- 6.4     11.2    6.9     38.9    -------
========== 8 / 20 (eta: 1587.4 / 3968.4 s) =========
------- 106.3   106.2   104.7   107.4   -------
========== 9 / 20 (eta: 2012.0 / 4471.0 s) =========
------- 4.7     5.1     7.4     105.1   -------
========== 10 / 20 (eta: 2134.3 / 4268.6 s) =====

# obj1 obs7

In [9]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_1c_obs7"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

### test

In [10]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 16.3    80.7    6.0     6.6     -------
========== 1 / 20 (eta: 109.6 / 2191.3 s) =========
------- 9.4     10.8    6.2     19.9    -------
========== 2 / 20 (eta: 155.9 / 1558.9 s) =========
------- 8.9     26.8    6.1     17.3    -------
========== 3 / 20 (eta: 215.0 / 1433.7 s) =========
------- 20.5    110.9   63.3    85.6    -------
========== 4 / 20 (eta: 495.3 / 2476.5 s) =========
------- 12.5    62.0    5.5     6.6     -------
========== 5 / 20 (eta: 582.0 / 2328.0 s) =========
------- 17.1    16.3    6.4     108.9   -------
========== 6 / 20 (eta: 730.7 / 2435.7 s) =========
------- 15.0    10.1    5.5     5.7     -------
========== 7 / 20 (eta: 766.9 / 2191.1 s) =========
------- 28.2    18.1    14.2    106.2   -------
========== 8 / 20 (eta: 933.6 / 2333.9 s) =========
------- 17.4    104.5   79.0    111.7   -------
========== 9 / 20 (eta: 1246.2 / 2769.3 s) =========
------- 104.4   11.8    91.7    15.6    -------
========== 10 / 20 (eta: 1469.7 / 2939.4 s) =======

# obj3 obs3

In [11]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_3c_obs3"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

### test

In [12]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 13.2    13.4    5.9     40.8    -------
========== 1 / 20 (eta: 73.3 / 1465.8 s) =========
------- 20.2    7.4     13.8    105.1   -------
========== 2 / 20 (eta: 219.8 / 2198.0 s) =========
------- 113.1   107.8   34.8    24.7    -------
========== 3 / 20 (eta: 500.2 / 3335.0 s) =========
------- 5.8     109.0   6.0     23.1    -------
========== 4 / 20 (eta: 644.1 / 3220.6 s) =========
------- 6.6     31.8    5.7     6.5     -------
========== 5 / 20 (eta: 694.7 / 2778.8 s) =========
------- 15.3    20.1    6.3     9.5     -------
========== 6 / 20 (eta: 745.8 / 2485.9 s) =========
------- 5.4     12.6    84.1    6.1     -------
========== 7 / 20 (eta: 853.9 / 2439.6 s) =========
------- 40.2    10.9    5.9     7.0     -------
========== 8 / 20 (eta: 917.8 / 2294.6 s) =========
------- 15.0    36.5    6.1     29.6    -------
========== 9 / 20 (eta: 1005.0 / 2233.3 s) =========
------- 65.8    7.1     6.5     23.1    -------
========== 10 / 20 (eta: 1107.5 / 2215.0 s) ========

# obj3 obs5

In [13]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_3c_obs5"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

### test

In [14]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 117.3   120.2   289.3   105.5   -------
========== 1 / 20 (eta: 632.2 / 12644.6 s) =========
------- 72.7    113.3   25.9    9.5     -------
========== 2 / 20 (eta: 853.7 / 8536.8 s) =========
------- 8.7     23.1    6.4     13.4    -------
========== 3 / 20 (eta: 905.3 / 6035.6 s) =========
------- 6.3     6.3     5.9     30.0    -------
========== 4 / 20 (eta: 953.9 / 4769.6 s) =========
------- 24.0    6.5     5.4     18.6    -------
========== 5 / 20 (eta: 1008.5 / 4033.9 s) =========
------- 112.7   17.4    28.2    41.5    -------
========== 6 / 20 (eta: 1208.2 / 4027.3 s) =========
------- 80.5    20.0    7.4     65.9    -------
========== 7 / 20 (eta: 1382.0 / 3948.4 s) =========
------- 28.0    24.4    17.2    34.3    -------
========== 8 / 20 (eta: 1485.9 / 3714.7 s) =========
------- 31.4    20.8    7.0     6.2     -------
========== 9 / 20 (eta: 1551.3 / 3447.4 s) =========
------- 83.8    6.9     18.7    6.8     -------
========== 10 / 20 (eta: 1667.6 / 3335.1 s) ==

# obj3 obs7

In [15]:
RTYPE = "indy7gripper"
FILE_OPTION = "obj_3c_obs7"
I_START = 0
I_END = 20
N_DAT = I_END-I_START

### test

In [16]:
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, RTYPE)
ROBOT_RESULT_ROOT = os.path.join(TEST_RESULT_PATH, RTYPE)
DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DAT_DIR)
RESULTSET_PATH = os.path.join(ROBOT_RESULT_ROOT, DAT_DIR)
file_list = sorted(os.listdir(DATASET_PATH))

gtimer = GlobalTimer.instance()
gtimer.reset(scale=1, timeunit='s')

gtimer.tic("full_loop")
for fid in range(I_START, I_END):
    e_step_list = []
    for i_c, cname in enumerate(CNAME_LIST):
        gtimer.tic("step")
        p = subprocess.Popen(['python', '{}src/scripts/developing/pddlstream/test_saved_scene.py'.format(RNB_PLANNING_DIR),
                              '--rtype', RTYPE,
                              '--dat_root', DAT_ROOT,
                              '--res_root', RES_ROOT,
                              '--dat_dir', DAT_DIR,
                              '--file_option', FILE_OPTION,
                              '--data_idx', str(fid),
                              '--cname', cname,
                              '--VISUALIZE', str(VISUALIZE),
                              '--PLAY_RESULT', str(PLAY_RESULT),
                              '--TIMEOUT_MOTION', str(TIMEOUT_MOTION),
                              '--MAX_TIME', str(MAX_TIME),
                              '--MAX_ITER', str(MAX_ITER), 
                              '--GRASP_SAMPLE', str(GRASP_SAMPLE),
                              '--STABLE_SAMPLE', str(STABLE_SAMPLE),
                              '--SEARCH_SAMPLE_RATIO', 
                              str(SEARCH_SAMPLE_RATIO),
                              '--MAX_SKELETONS', 
                              str(MAX_SKELETONS)])
        p.wait()
        e_step = np.round(gtimer.toc("step"), 1)
        e_step_list.append(e_step)
        print("------- " + ("{:<8}"*len(e_step_list)).format(*e_step_list) + "-------", end="\r")
        
    print("")
    print("========== {} / {} (eta: {} / {} s) =========".format(
        fid+1, N_DAT, *np.round(gtimer.eta("full_loop", fid-I_START+1, N_DAT), 1)))

------- 6.4     10.5    6.4     7.6     -------
========== 1 / 20 (eta: 30.9 / 617.1 s) =========
------- 32.8    41.2    5.8     33.3    -------
========== 2 / 20 (eta: 144.0 / 1439.8 s) =========
------- 7.1     102.4   108.0   6.3     -------
========== 3 / 20 (eta: 367.8 / 2452.2 s) =========
------- 108.8   8.2     109.2   8.5     -------
========== 4 / 20 (eta: 602.6 / 3012.8 s) =========
------- 25.8    31.0    15.8    26.6    -------
========== 5 / 20 (eta: 701.8 / 2807.2 s) =========
------- 72.3    20.5    8.8     55.4    -------
========== 6 / 20 (eta: 858.8 / 2862.7 s) =========
------- 13.4    13.0    26.6    108.2   -------
========== 7 / 20 (eta: 1020.1 / 2914.5 s) =========
------- 109.0   108.0   10.5    110.6   -------
========== 8 / 20 (eta: 1358.2 / 3395.5 s) =========
------- 99.8    35.4    7.1     10.4    -------
========== 9 / 20 (eta: 1510.9 / 3357.6 s) =========
------- 117.9   18.5    5.8     32.3    -------
========== 10 / 20 (eta: 1685.4 / 3370.8 s) =======